# Jupyter Notebook

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

# Column Meanings
- Rank_x - Ranking after season started/ended 
- Bye - number of weeks the team don't play 
- Fantrax,MFL,FFC - Each respective football fantasy league providing companies rankings  
- Rank_y - Ranking before season started
- Avg - ADP 

In [4]:
#overall adp files
overall_adp_2019 = pd.read_csv('data/overall_adp_rankings_2019.csv')
display(overall_adp_2019)
overall_adp_2022 = pd.read_csv('data/FantasyPros_2022_Overall_ADP_Rankings.csv')
display(overall_adp_2022)

,Rank,Player,Team,Bye,POS,MFL,Fantrax,RTSports,FFC,Sleeper,AVG
0,1.0,Saquon Barkley,NYG,11,RB1,1.0,1.0,NaN,1.0,NaN,1.0
1,2.0,Christian McCaffrey,CAR,7,RB2,3.0,2.0,NaN,2.0,NaN,2.5
2,3.0,Alvin Kamara,NO,9,RB3,2.0,3.0,NaN,3.0,NaN,2.5
3,4.0,Ezekiel Elliott,DAL,8,RB4,4.0,4.0,NaN,4.0,NaN,4.0
4,5.0,DeAndre Hopkins,ARI,12,WR1,5.0,6.0,NaN,5.0,NaN,5.3
...,...,...,...,...,...,...,...,...,...,...,...
400,401.0,Foster Moreau,LV,NaN,TE52,NaN,403.0,NaN,NaN,NaN,403.0
401,402.0,Bryce Love,NaN,NaN,RB109,NaN,408.0,NaN,NaN,NaN,408.0
402,403.0,D'Onta Foreman,CAR,7,RB110,NaN,409.0,NaN,NaN,NaN,409.0
403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Rank,Player,Team,Bye,POS,ESPN,RTSports,MFL,Fantrax,FFC,Sleeper,AVG
0,1.0,Jonathan Taylor,IND,14.0,RB1,1.0,1.0,NaN,NaN,NaN,1.0,1.0
1,2.0,Christian McCaffrey,CAR,13.0,RB2,2.0,2.0,NaN,NaN,NaN,2.0,2.0
2,3.0,Austin Ekeler,LAC,8.0,RB3,4.0,3.0,NaN,NaN,NaN,4.0,3.8
3,4.0,Cooper Kupp,LAR,7.0,WR1,3.0,5.0,NaN,NaN,NaN,5.0,4.0
4,5.0,Derrick Henry,TEN,6.0,RB4,5.0,8.0,NaN,NaN,NaN,3.0,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
350,351.0,Malik Willis,TEN,6.0,QB39,NaN,NaN,NaN,NaN,NaN,NaN,297.0
351,352.0,Joey Slye,WAS,14.0,K29,NaN,NaN,NaN,NaN,NaN,NaN,299.0
352,353.0,O.J. Howard,HOU,6.0,TE44,NaN,NaN,NaN,NaN,NaN,NaN,300.0
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#wide receiver stats
wr_stats_2018 = pd.read_csv('data/fantasy_pros_2018_wr_statistics.csv')
display(wr_stats_2018)

,Rank,Player,G,REC,YDS,Y/R,YBC,YBC/R,AIR,AIR/R,...,% TM,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,LNG
0,1.0,Tyreek Hill (MIA),16.0,87.0,"1,479",17.0,952,10.9,952,10.9,...,27.1%,94.0,6.0,9.0,50.0,27.0,16.0,8.0,4.0,75.0
1,2.0,Antonio Brown (FA),15.0,104.0,"1,297",12.5,812,7.8,812,7.8,...,24.9%,107.0,3.0,12.0,48.0,17.0,8.0,6.0,2.0,78.0
2,3.0,Davante Adams (LV),15.0,111.0,"1,386",12.5,911,8.2,911,8.2,...,27.5%,116.0,5.0,15.0,53.0,18.0,8.0,5.0,2.0,57.0
3,4.0,DeAndre Hopkins (ARI),16.0,115.0,"1,572",13.7,"1,185",10.3,"1,185",10.3,...,33.0%,121.0,2.0,10.0,66.0,23.0,9.0,4.0,0.0,49.0
4,5.0,Julio Jones (TB),16.0,113.0,"1,677",14.8,"1,229",10.9,"1,229",10.9,...,28.1%,123.0,9.0,13.0,78.0,24.0,10.0,2.0,1.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,182.0,Josh Malone (PIT),9.0,1.0,12,12.0,12,12.0,12,12.0,...,0.9%,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0
182,183.0,Victor Bolden Jr. (FA),6.0,1.0,10,10.0,9,9.0,9,9.0,...,0.8%,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,10.0
183,184.0,Jeremy Kerley (FA),1.0,2.0,7,3.5,-8,-4.0,-8,-4.0,...,0.7%,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#end of season wide receiver rankings
wr_rankings_2019 = pd.read_csv('data/wr_eos_rankings_2019.csv')
display(wr_rankings_2019)

,Rank,Player,Team,Points,Games,Avg
0,1.0,Michael Thomas,NO,374.6,16.0,23.4
1,2.0,Chris Godwin,TB,276.1,14.0,19.7
2,3.0,Julio Jones,ATL,274.1,15.0,18.3
3,4.0,Cooper Kupp,LAR,270.5,16.0,16.9
4,5.0,DeAndre Hopkins,HOU,269.5,15.0,18.0
...,...,...,...,...,...,...
289,290.0,Quincy Enunwa,NYJ,-1.4,1.0,-1.4
290,291.0,Greg Dortch,Multi,-2.0,2.0,-1.0
291,291.0,Ray-Ray McCloud,Multi,-2.0,6.0,-0.3
292,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#end of season overall rankings 
overall_rankings_2019 = pd.read_csv('data/FantasyPros_Fantasy_Football_Points_PPR.csv')
display(overall_rankings_2019.head(5))

,Rank,Player,Team,Position,Points,Games,Avg
0,1.0,Christian McCaffrey,CAR,RB,471.2,16.0,29.5
1,2.0,Lamar Jackson,BAL,QB,421.7,15.0,28.1
2,3.0,Michael Thomas,NO,WR,374.6,16.0,23.4
3,4.0,Dak Prescott,DAL,QB,348.9,16.0,21.8
4,5.0,Jameis Winston,TB,QB,335.2,16.0,21.0


In [8]:
#merged dataset with all types of players in 2019 
overall_merged_data = overall_adp_2019.merge(overall_rankings_2019, how= 'outer', on = 'Player')
overall_merged_data.head(5)

# get dataset for all players with all the positions 
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

,Rank_x,Player,Team_x,Bye,POS,MFL,Fantrax,RTSports,FFC,Sleeper,AVG,Rank_y,Team_y,Position,Points,Games,Avg
0,1.0,Saquon Barkley,NYG,11,RB1,1.0,1.0,NaN,1.0,NaN,1.0,41.0,NYG,RB,244.1,13.0,18.8
1,2.0,Christian McCaffrey,CAR,7,RB2,3.0,2.0,NaN,2.0,NaN,2.5,1.0,CAR,RB,471.2,16.0,29.5
2,3.0,Alvin Kamara,NO,9,RB3,2.0,3.0,NaN,3.0,NaN,2.5,37.0,NO,RB,248.5,14.0,17.8
3,4.0,Ezekiel Elliott,DAL,8,RB4,4.0,4.0,NaN,4.0,NaN,4.0,9.0,DAL,RB,311.7,16.0,19.5
4,5.0,DeAndre Hopkins,ARI,12,WR1,5.0,6.0,NaN,5.0,NaN,5.3,23.0,HOU,WR,269.5,15.0,18.0


In [9]:
#cleaned overall_merged_data df
# dropped 'RTSports' and 'Sleeper' column - Had a lot of NaN values 
# dropped 'MFL','Fantrax', and 'FFC'
# dropped 'POS', Team_y - duplicated from merging 
overall_2019_merged_data_clean = overall_merged_data.drop(columns=['RTSports', 'Sleeper', 'MFL', 'Fantrax', 'FFC', 'POS','Team_y'])
overall_2019_merged_data_clean.head(50)

# renaming column 
overall_2019_merged_data_clean.rename(columns = {"Rank_x": "Pre-season Rk","Team_x":"Team", "Rank_y": "Post-season Rk", "AVG": "ADP", "Avg": "PPG"})


,Pre-season Rk,Player,Team,Bye,ADP,Post-season Rk,Position,Points,Games,PPG
0,1.0,Saquon Barkley,NYG,11,1.0,41.0,RB,244.1,13.0,18.8
1,2.0,Christian McCaffrey,CAR,7,2.5,1.0,RB,471.2,16.0,29.5
2,3.0,Alvin Kamara,NO,9,2.5,37.0,RB,248.5,14.0,17.8
3,4.0,Ezekiel Elliott,DAL,8,4.0,9.0,RB,311.7,16.0,19.5
4,5.0,DeAndre Hopkins,ARI,12,5.3,23.0,WR,269.5,15.0,18.0
...,...,...,...,...,...,...,...,...,...,...
848,NaN,Jarrett Stidham,NaN,NaN,NaN,802.0,QB,-0.6,3.0,-0.2
849,NaN,Darrius Shepherd,NaN,NaN,NaN,804.0,WR,-0.9,6.0,-0.2
850,NaN,Quincy Enunwa,NaN,NaN,NaN,805.0,WR,-1.4,1.0,-1.4
851,NaN,Greg Dortch,NaN,NaN,NaN,806.0,WR,-2.0,2.0,-1.0


In [10]:
#merged dataset of Wide Receivers in 2019 
#more data analysis - several models to be used 
#what other analysis can we do 
# define variable to address NaN values 



In [11]:
#merged dataset of Running Back in 2019 

In [12]:
#merged dataset of Quarter Back in 2019 

In [13]:
# merge dataset if clean; clean as much as data possible; remove data files; 
#nyc opendata - for examples of clean dataset
#prediction model on which player is going to be a breakout player
#cluster of everyone else; cluster of busts; cluster of breakout players
# another column that indicates that they are breakout players 
# https://www.kaggle.com/code/davidvilensky/mf-and-etfs-analysis/notebook - example to reference to 